As a quick refresher, the Academy Awards, also known as the Oscars, is an annual awards ceremony hosted to recognize the achievements in the film industry. There are many different awards categories and the members of the academy vote every year to decide which artist or film should get the award. Each row in our data represents a nomination for an award. Recall that our database file, nominations.db, contains just the nominations table. This table has the following schema:

Year - the year of the awards ceremony, integer type.
Category - the category of award the nominee was nominated for, text type.
Nominee - the person nominated for the award, text type.
Movie - the movie the nominee participated in, text type.
Character - the name of the character the nominee played, text type.
Won - if this nominee won the award, integer type.
Let's now set up our enviroment and spend some time getting familiar with the data before we start normalizing it.


In [5]:
import sqlite3

In [6]:
conn = sqlite3.Connection('nominations.db')

In [10]:
schema = conn.execute("pragma table_info(nominations);").fetchall()

In [12]:
#print(schema)
for s in schema:
    print(s)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [9]:
first_ten = conn.execute("select * from nominations limit 10;").fetchall()

In [13]:
for entry in first_ten:
    print(entry)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


# 2: Creating The Ceremonies Table

In [14]:
years_hosts = [(2010, "Steve Martin"),
               (2009, "Hugh Jackman"),
               (2008, "Jon Stewart"),
               (2007, "Ellen DeGeneres"),
               (2006, "Jon Stewart"),
               (2005, "Chris Rock"),
               (2004, "Billy Crystal"),
               (2003, "Steve Martin"),
               (2002, "Whoopi Goldberg"),
               (2001, "Steve Martin"),
               (2000, "Billy Crystal"),
            ]

In [15]:
create_ceremonies = "create table ceremonies (id integer primary key, year integer, host text);"
conn.execute(create_ceremonies)
insert_query = "insert into ceremonies (Year, Host) values (?,?);"
conn.executemany(insert_query, years_hosts)

In [19]:
first_ten_ceremonies = conn.execute("select * from ceremonies limit 10;").fetchall()

for c in first_ten_ceremonies:
    print(c)

(1, 2010, 'Steve Martin')
(2, 2009, 'Hugh Jackman')
(3, 2008, 'Jon Stewart')
(4, 2007, 'Ellen DeGeneres')
(5, 2006, 'Jon Stewart')
(6, 2005, 'Chris Rock')
(7, 2004, 'Billy Crystal')
(8, 2003, 'Steve Martin')
(9, 2002, 'Whoopi Goldberg')
(10, 2001, 'Steve Martin')


In [20]:
pragma_ceremonies = conn.execute("pragma table_info(ceremonies);").fetchall()

for p in pragma_ceremonies:
    print(p)

(0, 'id', 'integer', 0, None, 1)
(1, 'year', 'integer', 0, None, 0)
(2, 'host', 'text', 0, None, 0)


# Foreign key constraints

In [21]:
conn.execute("PRAGMA foreign_keys = ON;")

# Setting up one-to-many

In [22]:
create_nominations_two = '''create table nominations_two 
(id integer primary key, 
category text, 
nominee text, 
movie text, 
character text, 
won text,
ceremony_id integer,
foreign key(ceremony_id) references ceremonies(id));
'''

In [23]:
nom_query = '''
select ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies 
on nominations.year == ceremonies.year
;
'''

In [25]:
joined_nominations = conn.execute(nom_query).fetchall()

In [26]:
print(joined_nominations)

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0), (1, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0), (1, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0), (1, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1), (1, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', 0), (1, 'Actor -- Supporting Role', 'Christian Bale', 'The Fighter', 'Dicky Eklund', 1), (1, 'Actor -- Supporting Role', 'John Hawkes', "Winter's Bone", 'Teardrop', 0), (1, 'Actor -- Supporting Role', 'Jeremy Renner', 'The Town', 'James Coughlin', 0), (1, 'Actor -- Supporting Role', 'Mark Ruffalo', 'The Kids Are All Right', 'Paul', 0), (1, 'Actor -- Supporting Role', 'Geoffrey Rush', "The King's Speech", 'Lionel Logue', 0), (1, 'Actress -- Leading Role', 'Annette Bening', 'The Kids Are All Right', 'Nic', 0), (1, 'Actress -- Leading Role', 'Nicole Kidman', 'Rabbit Hole', 'Becca', 0), (1,

In [27]:
conn.execute(create_nominations_two)

In [28]:
insert_nominations_two = '''insert into nominations_two (ceremony_id, category, nominee, movie, character, won) 
values (?,?,?,?,?,?);
'''

In [29]:
conn.executemany(insert_nominations_two, joined_nominations)

In [30]:
print(conn.execute("select * from nominations_two limit 5;").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', '0', 1), (2, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', '0', 1), (3, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', '0', 1), (4, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', '1', 1), (5, 'Actor -- Leading Role', 'James Franco', '127 Hours', 'Aron Ralston', '0', 1)]


# Deleting and renaming tables

In [31]:
drop_nominations = "drop table nominations;"
conn.execute(drop_nominations)

In [32]:
rename_nominations_two = "alter table nominations_two rename to nominations;"
conn.execute(rename_nominations_two)

# Creating joined table

In [34]:
create_movies = "create table movies (id integer primary key,movie text);"
create_actors = "create table actors (id integer primary key,actor text);"
create_movies_actors = '''create table movies_actors (id INTEGER PRIMARY KEY,
movie_id INTEGER references movies(id), actor_id INTEGER references actors(id));
'''
conn.execute(create_movies)
conn.execute(create_actors)
conn.execute(create_movies_actors)

# Populating the movies and actors tables

In [37]:
insert_movies = "insert into movies (movie) select distinct movie from nominations;"
insert_actors = "insert into actors (actor) select distinct nominee from nominations;"
conn.execute(insert_movies)
conn.execute(insert_actors)

In [38]:
movies = conn.execute("select * from movies limit 5;").fetchall()
for movie in movies:
    print(movie)

(1, 'Biutiful')
(2, 'True Grit')
(3, 'The Social Network')
(4, "The King's Speech")
(5, '127 Hours')


In [39]:
actors = conn.execute("select * from actors limit 5;").fetchall()
for actor in actors:
    print(actor)

(1, 'Javier Bardem')
(2, 'Jeff Bridges')
(3, 'Jesse Eisenberg')
(4, 'Colin Firth')
(5, 'James Franco')
